In [1]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [35]:
import numpy as np
import pandas as pd
import datetime as dt
from sqlalchemy import and_, func, inspect

# Reflect Tables into SQLAlchemy ORM

In [3]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [4]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///resources/hawaii.sqlite")

In [5]:
# reflect an existing database into a new model
Base = automap_base()

# reflect the tables
Base.prepare(engine, reflect=True)

In [6]:
# View all of the classes that automap found
Base.classes.keys()

['measurement', 'station']

In [7]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [8]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Precipitation Analysis

In [9]:
# Find the most recent date in the data set.
most_recent = session.query(Measurement.date).order_by(Measurement.date.desc()).first()
most_recent

('2017-08-23')

In [30]:
# Calculate the date one year from the last date in data set.
one_year = dt.date(2017, 8, 23) - dt.timedelta(days=365)
print('Query Date:', one_year)

Query Date: 2016-08-23


In [54]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 
# Perform a query to retrieve the data and precipitation scores
date_prcp = engine.execute('SELECT date,prcp FROM Measurement').fetchall()
date_prcp
dateprcp_rows = [{"date": result[0], "prcp": result[1]} for result in date_prcp]
dateprcp_rows

[{'date': '2010-01-01', 'prcp': 0.08},
 {'date': '2010-01-02', 'prcp': 0.0},
 {'date': '2010-01-03', 'prcp': 0.0},
 {'date': '2010-01-04', 'prcp': 0.0},
 {'date': '2010-01-06', 'prcp': None},
 {'date': '2010-01-07', 'prcp': 0.06},
 {'date': '2010-01-08', 'prcp': 0.0},
 {'date': '2010-01-09', 'prcp': 0.0},
 {'date': '2010-01-10', 'prcp': 0.0},
 {'date': '2010-01-11', 'prcp': 0.01},
 {'date': '2010-01-12', 'prcp': 0.0},
 {'date': '2010-01-14', 'prcp': 0.0},
 {'date': '2010-01-15', 'prcp': 0.0},
 {'date': '2010-01-16', 'prcp': 0.0},
 {'date': '2010-01-17', 'prcp': 0.0},
 {'date': '2010-01-18', 'prcp': 0.0},
 {'date': '2010-01-19', 'prcp': 0.0},
 {'date': '2010-01-20', 'prcp': 0.0},
 {'date': '2010-01-21', 'prcp': 0.0},
 {'date': '2010-01-22', 'prcp': 0.0},
 {'date': '2010-01-23', 'prcp': 0.0},
 {'date': '2010-01-24', 'prcp': 0.01},
 {'date': '2010-01-25', 'prcp': 0.0},
 {'date': '2010-01-26', 'prcp': 0.04},
 {'date': '2010-01-27', 'prcp': 0.12},
 {'date': '2010-01-28', 'prcp': 0.0},
 {'da

In [ ]:
# Save the query results as a Pandas DataFrame and set the index to the date column
# Sort the dataframe by date
# Use Pandas Plotting with Matplotlib to plot the data
pd.DataFrame(dateprcp_rows).set_index("date").plot(kind = 'bar')
plt.title("Precipitation 8/23/16-8/23/17")
plt.show()

In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data
dateprcp_rows.describe()

# Exploratory Station Analysis

In [14]:
# Design a query to calculate the total number stations in the dataset


In [15]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.


In [16]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.


In [17]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram


# Close session

In [18]:
# Close Session
session.close()